In [2]:
import os
import glob
import pandas as pd
import re
import jieba    
import json

In [3]:
# JSON 파일을 읽고 필요한 데이터를 추출하는 함수
def extract_data_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    standard_forms = []
    dialect_forms = []
    
    for utterance in data['utterance']:
        standard_forms.append(utterance['standard_form'])
        dialect_forms.append(utterance['dialect_form'])
    
    return standard_forms, dialect_forms

In [4]:
# 현재 작업 디렉토리 설정
path = os.getcwd()

# Training 및 Validation 경로 설정
train_path = os.path.join(path, 'Training', '*.json')
valid_path = os.path.join(path, 'Validation', '*.json')

# Training 및 Validation 파일 목록 가져오기
train_files = glob.glob(train_path)
valid_files = glob.glob(valid_path)

# 데이터를 저장할 리스트 초기화
x_train, y_train, x_valid, y_valid = [], [], [], []

# Training 파일에서 데이터 추출
for t in train_files:
    std_forms, dial_forms = extract_data_from_json(t)
    x_train.extend(std_forms)
    y_train.extend(dial_forms)

# Validation 파일에서 데이터 추출
for v in valid_files:
    std_forms, dial_forms = extract_data_from_json(v)
    x_valid.extend(std_forms)
    y_valid.extend(dial_forms)

# 학습용 데이터를 pandas DataFrame으로 변환
train_df = pd.DataFrame({'표준어': x_train, '제주어': y_train})

# 검증용 데이터를 pandas DataFrame으로 변환
valid_df = pd.DataFrame({'표준어': x_valid, '제주어': y_valid})

In [5]:

# 데이터 정보 출력
print('번역 데이터 : 학습용 {}개, 검증용 {}개'.format(len(train_df), len(valid_df)))
print(train_df.sample(10))  # 학습용 데이터에서 임의의 5개 샘플 출력

번역 데이터 : 학습용 2774257개, 검증용 333802개
                                         표준어  \
145368         그렇게 하다가 수도 안 나오는 날은 아무튼 여기 사람   
1694774  점심 지도 안 하지 않애? 나 그 때 점심 지도 안 들어갔는데?   
2617302                                    음   
1560870                와 에르메스 카드지갑 사십삼만 원 엥?   
1142034                                   어휴   
194226                                     다   
1828401                      막 나이 많은 사람들만 가.   
2698119                                 신가하네   
2185337                                    응   
961536      학교 -어디- 어디야? 막 이래 가지고 아 저 한라대학요.   

                                         제주어  
145368            경 허당 수도 안 나오는 날은 아무튼 요기 사람  
1694774  점심 지도 안 하지 않애? 나 그 때 점심 지도 안 들어갔는데?  
2617302                                    음  
1560870                와 에르메스 카드지갑 사십삼만 원 엥?  
1142034                                   어휴  
194226                                     다  
1828401                      막 나이 많은 사람들만 가.  
2698119                                 신기한게  
2185337      

In [6]:
# 필요에 따라 DataFrame을 CSV 파일로 저장
train_df.to_csv('train_data.csv', index=False, encoding='utf-8')
valid_df.to_csv('valid_data.csv', index=False, encoding='utf-8')

In [7]:
def preprocess_kr(w):
    w = re.sub(r"([?'!¿\-·\"])", r" \1 ", w)
    w = re.sub(r'[ |ㄱ-ㅎ|ㅏ-ㅣ]+', r" ", w)
    w = re.sub(r"\,(?=[0-9])", r"", w)
    w = w[:-1].strip()
    w = '<start> ' + w + ' <end>'
    return w  

In [8]:
# 표준어와 제주어 데이터를 전처리하는 함수
def preprocess(path):
    files = glob.glob(os.path.join(path, '*.json'))
    std, jej = [], []  # 빈 리스트 초기화
    
    for f in files:
        std_forms, dial_forms = extract_data_from_json(f)
        std.extend(std_forms)
        jej.extend(dial_forms)
    
    std_series = pd.Series(std)
    jej_series = pd.Series(jej)
    
    df = pd.concat([std_series, jej_series], axis=1)
    df.columns = ['표준어', '제주어']
    
    df['표준어'] = df['표준어'].apply(preprocess_kr)
    df['제주어'] = df['제주어'].apply(preprocess_kr)
    
    return df

In [9]:
# 현재 작업 디렉토리 설정 및 데이터 전처리
path = os.getcwd() + '\\Training'
train_df = preprocess(path)

print('훈련 데이터셋 크기 : {}'.format(train_df.shape))
print(train_df.sample(3))

훈련 데이터셋 크기 : (2774257, 2)
                                           표준어  \
1587307  <start> 애기 옷 진짜 애기 옷 하고 싶었는데 애들 <end>   
2565260           <start> 맞아 맞아 맞아 완전 어려 <end>   
1771793               <start> 동생이 싸한 느낌이 <end>   

                                           제주어  
1587307  <start> 애기 옷 진짜 애기 옷 하고 싶었는데 애들 <end>  
2565260           <start> 맞아 맞아 맞아 완전 어려 <end>  
1771793               <start> 동생이 싸한 느낌이 <end>  


In [10]:
for i in range(10):
    print(train_df.iloc[i,0])
    print(train_df.iloc[i,1])
    print()

<start> 난 진짜 벌초 할때 뱀나오잖아 뱀 <end>
<start> 난 진짜 벌초 할때 뱀나오잖아 뱀 <end>

<start> 어어 그니 <end>
<start> 어어 그니 <end>

<start> 아 잘 <end>
<start> 아 잘 <end>

<start> 예초기 뱀도 짤 <end>
<start> 예초기 뱀도 짤 <end>

<start> 어 그니까 완전 끔찍 <end>
<start> 어 그니까 완전 끔찍 <end>

<start> 그니 <end>
<start> 그니 <end>

<start> 예전에 그렇 <end>
<start> 예전에 <end>

<start> 해놨잖아 작은오 <end>
<start> 해놨잖아 작은오 <end>

<start>  <end>
<start>  <end>

<start> 아 예초하다가(이)/(#이) 뱀 나온거야 뱀이 하나 도 아니 <end>
<start> 아 예초하다가(이)/(#이) 뱀 나온거야 뱀이 하나 도 아니 <end>



In [11]:
rows_per_file = len(train_df) // 4  # 데이터의 약 1/3씩 저장하기 위해 나눔

# 데이터프레임을 분할하고 각각을 저장
for i, chunk in enumerate(range(0, len(train_df), rows_per_file)):
    # 각 부분을 CSV 파일로 저장
    train_df.iloc[chunk:chunk + rows_per_file].to_csv(f'train_df_part{i+1}.csv', index=False)

print("파일 분할 저장 완료")

파일 분할 저장 완료


In [12]:
valid_df.to_csv('val_df.csv')